In [ ]:
MODEL_NAME = "peanuts"  # Model name
DATASET_NAME = "peanuts"  # Dataset name

### SETUP

In [ ]:
import random
from dotenv import load_dotenv
import os
from pprint import pprint
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from focoos.ports import Hyperparameters
from supervision import plot_image
from utils import display_instseg, display_detection
from dataset import get_dataset

load_dotenv()

In [ ]:
dataset_cfg = get_dataset(DATASET_NAME)  # Get dataset configuration

DATASET_NAME = dataset_cfg["name"]  # Dataset name
VIS_FUNC = (
    display_detection if dataset_cfg["task"] == "detection" else display_instseg
)  # Visualization function
FOCOOS_MODEL = (
    "focoos_object365"
    if dataset_cfg["task"] == "detection"
    else "focoos_ade_ins_medium"
)  # Start model

PATH = dataset_cfg["path"] + "/valid"  # Path to the dataset (for visualization only)
PATHS = [os.path.join(PATH, p) for p in os.listdir(PATH)]

API_KEY = os.getenv("FOCOOS_API_KEY")
HOST = FocoosEnvHostUrl.DEV

## Visualize the dataset


In [ ]:
if dataset_cfg["task"] == "instance_segmentation":
    display_instseg(PATH, num_images=5, annotate=True)
else:
    display_detection(PATH, num_images=5, annotate=True)

# Training and Inferencing with Focoos 

In [ ]:
focoos = Focoos(api_key=API_KEY, host_url=HOST)

## Deploy the model

Use the model on Focoos servers

In [ ]:
model = focoos.get_model_by_name(MODEL_NAME)

output, preview = model.infer(
    PATHS[random.randint(0, len(PATHS) - 1)], threshold=0.4, annotate=True
)

plot_image(preview)

Use the model locally (first download the model then use it)

In [ ]:
local_model = focoos.get_local_model(model.metadata.ref)

In [ ]:
output, preview = local_model.infer(
    PATHS[random.randint(0, len(PATHS) - 1)], annotate=True, threshold=0.4
)

pprint(output.latency)

pprint(output)
# plot_image(preview)

## Train a model for the dataset.

In [ ]:
model = focoos.new_model(
    name=MODEL_NAME + "_live",
    focoos_model=FOCOOS_MODEL,
    description=f"A model for the {DATASET_NAME} dataset",
)

dataset = focoos.get_dataset_by_name(DATASET_NAME)

res = model.train(
    dataset_ref=dataset.ref,  # Dataset reference
    hyperparameters=Hyperparameters(  # Hyperparameters for the training
        learning_rate=0.0005, batch_size=16, max_iters=1000
    ),
)

model.monitor_train()

# Finally, we can also run a gradio DEMO